## **Effortless Hyperparameters Tuning with Apache Spark**

In [2]:
!pip install pandas

     |████████████████████████████████| 12.2 MB 23.8 MB/s eta 0:00:01
     |████████████████████████████████| 498 kB 65.5 MB/s eta 0:00:01
     |████████████████████████████████| 17.1 MB 64.9 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
!pip install numpy

In [15]:
!pip install scikit-learn

     |████████████████████████████████| 31.2 MB 22.6 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 42.2 MB/s eta 0:00:01
     |████████████████████████████████| 33.8 MB 65.4 MB/s eta 0:00:01


In [23]:
!pip install xgboost

     |████████████████████████████████| 193.6 MB 167 kB/s  eta 0:00:01


In [25]:
!pip install interruptingcow

  Created wheel for interruptingcow: filename=interruptingcow-0.8-py3-none-any.whl size=5804 sha256=1a477818fd15c4fbb5cbfbb6cffb708d7111b61bd9d38d88376b7942785b22a2
  Stored in directory: /config/.cache/pip/wheels/ed/43/cc/2a5511c6cabcee2e975227e93fb05c708e406844b0ff24ea49
Successfully built interruptingcow


In [27]:
!pip install preprocessing

     |████████████████████████████████| 349 kB 16.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 55.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 66.8 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.2.4-py3-none-any.whl size=1367704 sha256=66a204aa505366b2e0acecea91726e6353e78e6c6adc7971f12925896741c0e3
  Stored in directory: /config/.cache/pip/wheels/d9/37/86/b5270b826e4b542bd6791005300c9d3864059901c7efc03545
Successfully built nltk


In [34]:
!pip install pymongo

     |████████████████████████████████| 518 kB 21.6 MB/s eta 0:00:01
     |████████████████████████████████| 269 kB 60.7 MB/s eta 0:00:01


In [36]:
!pip install joblibspark

In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [6]:
df = pd.read_csv("https://gist.githubusercontent.com/aialenti/22ee46bb7cfc714286c881cc556cb9f0/raw/395a57e5462a6d352319dda4e9cad11a5b07ab35/file.csv")

In [7]:
def preprocessing():
    # Read input data
    url = 'https://gist.githubusercontent.com/aialenti/22ee46bb7cfc714286c881cc556cb9f0/raw/395a57e5462a6d352319dda4e9cad11a5b07ab35/file.csv'
    train = pd.read_csv(url)
    categorical = ["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"]

    #   Convert categorical data
    for c in categorical:
        group_by = train.groupby(by=c)["y"].mean().reset_index().rename(columns={"y": "{}_converted".format(c)})
        train = pd.merge(train, group_by, how='inner', on=c)

    train = train.drop(categorical, axis=1)

    #   Drop the ID column
    X = train.drop("ID", axis=1).drop("y", axis=1)
    y = train["y"]
    return X, y

In [8]:
preprocessing()

(   X10  X11  X12  X13  X14  X15  X16  X17  X18  X19  ...  X384  X385  \
 0    0    0    0    1    0    0    0    0    1    0  ...     0     0   
 1    0    0    0    0    0    0    0    0    1    0  ...     0     0   
 2    0    0    0    0    0    0    0    1    0    0  ...     0     0   
 3    0    0    0    0    0    0    0    0    0    0  ...     0     0   
 4    0    0    0    0    0    0    0    0    0    0  ...     0     0   
 
    X0_converted  X1_converted  X2_converted  X3_converted  X4_converted  \
 0        109.67       104.415        130.81        130.81        90.848   
 1        109.67        84.575         88.53         88.53        90.848   
 2         78.30        76.260         78.30         76.26        90.848   
 3         78.30       104.415         78.30         79.32        90.848   
 4         78.30        84.575         78.30         79.32        90.848   
 
    X5_converted  X6_converted  X8_converted  
 0        130.81       103.535        109.67  
 1      

In [11]:
param_grid = {'learning_rate': [float(v) for v in np.arange(0.01, 0.25, 0.01)],
              'colsample_bytree': [float(v) for v in np.arange(0.8, 1.01, 0.1)],
              'subsample': [float(v) for v in np.arange(0.5, 1.01, 0.1)],
              'n_estimators': [int(v) for v in np.arange(100, 3000, 100)],
              'reg_alpha': [float(v) for v in np.arange(0.01, 0.5, 0.05)],
              'max_depth': [int(v) for v in np.arange(3, 14, 1)],
              'gamma': [int(v) for v in np.arange(0, 10, 2)]
              }

In [12]:
def get_grid_iterable():
    param_grid = {'learning_rate': [float(v) for v in np.arange(0.01, 0.25, 0.01)],
                  'colsample_bytree': [float(v) for v in np.arange(0.8, 1.01, 0.1)],
                  'subsample': [float(v) for v in np.arange(0.5, 1.01, 0.1)],
                  'n_estimators': [int(v) for v in np.arange(100, 3000, 100)],
                  'reg_alpha': [float(v) for v in np.arange(0.01, 0.5, 0.05)],
                  'max_depth': [int(v) for v in np.arange(3, 14, 1)],
                  'gamma': [int(v) for v in np.arange(0, 10, 2)]
                  }
    grid_iter = []
    length = 1
    for k in param_grid:
        grid_iter.append(param_grid[k])
        length *= len(param_grid[k])

    return itertools.product(*grid_iter), list(param_grid.keys()), length-1

In [20]:
def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

In [32]:
# Imports
import numpy as np
import pandas as pd
import xgboost as xgb
from interruptingcow import timeout
from sklearn.model_selection import KFold  # import KFold
from sklearn.metrics import r2_score
import json
# from preprocessing import preprocessing, xgb_r2_score # The preprocessing and the r2 evaluation
# from generate_grid import get_grid_iterable # The grid


def grid_search(timeout_seconds, cv_splits, boosting_rounds):
    #   Read input data
    X, y = preprocessing()

    #   Create dataframe to collect the results
    tests_columns = ["test_nr", "cv_mean", "cv_min", "cv_max", "cv_median", "params"]
    test_id = 0
    tests = pd.DataFrame(columns=tests_columns)

    #   Cross validation number of splits
    kf = KFold(n_splits=cv_splits)

    #   Execute until timeout occurs
    with timeout(timeout_seconds, exception=RuntimeError):

        #   Get the grid
        grid_iter, keys, length = get_grid_iterable()
        try:

            #   For every element of the grid
            for df_grid in grid_iter:
                #   Prepare a list to collect the scores
                score = []
                params = dict(zip(keys, df_grid))

                #   The objective function
                params["objective"] = "reg:squarederror"

                #   For each fold, train XGBoost and spit out the results
                for train_index, test_index in kf.split(X.values):

                    #   Get X train and X test
                    X_train, X_test = X.iloc[train_index], X.iloc[test_index]

                    #   Get y train and y test
                    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                    #   Convert into DMatrix
                    d_train = xgb.DMatrix(X_train, label=y_train)
                    d_valid = xgb.DMatrix(X_test, label=y_test)
                    d_test = xgb.DMatrix(X_test)
                    watchlist = [(d_train, 'train'), (d_valid, 'valid')]

                    #   Create the classifier using the current grid params. Apply early stopping of 50 rounds
                    clf = xgb.train(params, d_train, boosting_rounds, watchlist, early_stopping_rounds=50,
                                    feval=xgb_r2_score, maximize=True, verbose_eval=10)
                    y_hat = clf.predict(d_test)

                    #   Append Scores on the fold kept out
                    score.append(r2_score(y_test, y_hat))

                #   Store the result into a dataframe
                score_df = pd.DataFrame(columns=tests_columns, data=[
                    [test_id, np.mean(score), np.min(score), np.max(score), np.median(score),
                     json.dumps(dict(zip(keys, [str(g) for g in df_grid])))]])
                test_id += 1
                tests = pd.concat([tests, score_df])
        except RuntimeError:
            #   When timeout occurs an exception is raised and the main cycle is broken
            pass

    #   Spit out the results
    tests.to_csv("grid-search.csv", index=False)
    print(tests)


if __name__ == "__main__":
    grid_search(timeout_seconds=3600, cv_splits=4, boosting_rounds=500)

NameError: name 'itertools' is not defined

In [37]:
from interruptingcow import timeout
from sklearn.model_selection import KFold
import pymongo
from joblibspark import register_spark
from joblib import Parallel, delayed
from pprint import pprint
import random

In [38]:
def random_search_spark(timeout_seconds, cv_splits, boosting_rounds):
    #   Read and preprocess input data
    X, y = preprocessing()

    #   !!! First step to use Joblib Spark
    register_spark()  # register spark backend

    kf = KFold(n_splits=cv_splits)

    #   Init MongoDB
    client = pymongo.MongoClient(MONGO_IP, 27017)
    db = client[MONGO_DATABASE]
    collection = db[MONGO_COLLECTION]
    collection.drop()

    #   InterruptingCow will kill everything after the timeout
    with timeout(timeout_seconds, exception=RuntimeError):
        try:
            #   !!! The actual Joblib magic
            Parallel(backend="spark", n_jobs=NODES)(
                delayed(evaluate)(X, y, kf, boosting_rounds) for p in range(0, NODES))
        except RuntimeError:
            print("Runtime")
    db = client[MONGO_DATABASE]
    collection = db[MONGO_COLLECTION]

    cursor = collection.find({})
    for document in cursor:
        pprint(document)
    client.close()
    sc.stop()

In [39]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import r2_score
import pymongo
from joblib import Parallel, delayed

import random

In [40]:
#   "evaluate" is the function that is executed on the single Spark executor
def evaluate(X, y, kf, boosting_rounds):
    # We need an inner function because we want to use multithreading on each node
    def evaluate_inner(X, y, kf, boosting_rounds):
        score = []
        # !!! Pick a random point of the serach space - Conceptually is like running NODES distinct Random Search
        params = generate_random_configuration()
        params["objective"] = "reg:squarederror"
        print(params)
        for train_index, test_index in kf.split(X.values):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            d_train = xgb.DMatrix(X_train, label=y_train)
            d_valid = xgb.DMatrix(X_test, label=y_test)
            d_test = xgb.DMatrix(X_test)
            watchlist = [(d_train, 'train'), (d_valid, 'valid')]
            clf = xgb.train(params, d_train, boosting_rounds, watchlist, early_stopping_rounds=50,
                            feval=xgb_r2_score, maximize=True, verbose_eval=25)
            y_hat = clf.predict(d_test)
            score.append(r2_score(y_test, y_hat))

        output_dict = {
            'min_score': np.min(score),
            'max_score': np.max(score),
            'average_score': np.mean(score),
            'median_score': np.median(score),
            'params': params
        }

        client = pymongo.MongoClient(MONGO_IP, 27017)
        db = client[MONGO_DATABASE]
        collection = db[MONGO_COLLECTION]
        collection.insert_one(output_dict)
        client.close()
        return score

    # Same concept as seen in the Spark parallelization, but now we are using plain Joblib to parallelize
    # the instances in the chunk. In this case we are specifying how many cores we want to use (CORES variable)
    Parallel()(delayed(evaluate_inner)(X, y, kf, boosting_rounds) for i in range(0, 100000))


In [41]:
random_search_spark(timeout_seconds, cv_splits, boosting_rounds)

NameError: name 'timeout_seconds' is not defined